<a href="https://colab.research.google.com/github/MichaelHuang05/test-repo/blob/master/FintechDR_0_and_1_CNN_with_resample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

from google.colab import drive
drive.mount('/content/Drive')


path = "/content/Drive/My Drive/Colab Notebooks/images"
os.chdir(path) 
with open('/content/Drive/My Drive/Colab Notebooks/labels.json') as label:
    labels = json.load(label)

label_df = pd.DataFrame(labels, columns = ["file_name", "DR_stage"])
label_df["Eye_threaten"] = 0
label_df["Eye_threaten"][label_df["DR_stage"] > 1] = 1


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/Drive


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [0]:
#### training and testing set split
from sklearn.model_selection import train_test_split 
from keras.utils import to_categorical


y_train, y_test = train_test_split(label_df, test_size=0.2)

q = pd.DataFrame(y_train)
a = q[q["Eye_threaten"].isin([0])]
b = q[q["Eye_threaten"].isin([1])]

print(y_test.head())
q.groupby(['DR_stage']).agg({'DR_stage':"count"})
# d



Using TensorFlow backend.


             file_name  DR_stage  Eye_threaten
1373  37770_right.jpeg         0             0
920    37059_left.jpeg         0             0
1425   13673_left.jpeg         0             0
1119  25620_right.jpeg         3             1
1348  42149_right.jpeg         0             0


,DR_stage
DR_stage,
0,875
1,102
2,182
3,25
4,16


In [0]:
### we need to bootstrap to make each level to have the "balanced" amount of observations 
from random import sample

boota = np.random.choice(a["file_name"], replace = True, size = 600)
bootb = np.random.choice(b["file_name"], replace = True, size = 600)

### later use boot_df
boota_df = pd.DataFrame(boota, columns = ["file_name"])
bootb_df = pd.DataFrame(bootb, columns = ["file_name"])


train_filenames_df = pd.concat([boota_df, bootb_df])
y_new_train = pd.merge(train_filenames_df, label_df, how = "left", on = ["file_name", "file_name"])


# Onehot encoding for catogory
y_new_trainOneHot = to_categorical(y_new_train.iloc[:,2])
y_testOneHot = to_categorical(y_test.iloc[:,2])


In [0]:
# Train set
################################################

train_images = []
train_filenames_ls = list(train_filenames_df["file_name"])

for i in range(len(train_filenames_ls)):              
    temp = cv2.imread(path + "/" + train_filenames_ls[i])
    temp = cv2.resize(temp, (512,512))
    train_images.append(temp)

train_images_arr = np.array(train_images)

# Test set
#############################################

test_images = []
test_filenames_ls = list(y_test["file_name"])

for i in range(len(test_filenames_ls)):              
    temp = cv2.imread(path + "/" + test_filenames_ls[i])
    temp = cv2.resize(temp, (512,512))
    test_images.append(temp)

test_images_arr = np.array(test_images)


In [0]:
### CNN 
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D




model = Sequential()

model.add(Conv2D(filters = 10, kernel_size = (3,3), input_shape = (512,512,3), activation = "relu"))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(filters = 10, kernel_size = (3,3), activation = "relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters = 10, kernel_size = (3,3), activation = "relu"))
model.add(MaxPooling2D(pool_size=(2,2)))



model.add(Flatten())
model.add(Dense(64, activation = "relu"))

model.add(Dropout(0.2))

model.add(Dense(64, activation = "relu"))

model.add(Dropout(0.2))

model.add(Dense(64, activation = "relu"))

model.add(Dropout(0.2))

model.add(Dense(64, activation = "relu"))

model.add(Dropout(0.2))

model.add(Dense(64, activation = "relu"))

model.add(Dropout(0.2))


model.add(Dense(2, activation = 'softmax'))

model.compile(loss = "categorical_crossentropy",
             optimizer = "adam",
             metrics = ['accuracy'])

# maybe should change to focal loss?


#model = tf.contrib.tpu.keras_to_tpu_model(
#    model,
#    strategy=tf.contrib.tpu.TPUDistributionStrategy(
#        tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))



model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 510, 510, 10)      280       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 255, 255, 10)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 253, 253, 10)      910       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 126, 126, 10)      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 124, 124, 10)      910       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 62, 62, 10)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 38440)            

In [0]:
model.fit(x = train_images_arr, y = y_new_trainOneHot, batch_size = 100, epochs = 1000, validation_split = 0.1, verbose = 2)

Train on 1080 samples, validate on 120 samples
Epoch 1/1000
 - 5s - loss: 8.6093 - acc: 0.4472 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 2/1000
 - 5s - loss: 7.8687 - acc: 0.5102 - val_loss: 9.5657 - val_acc: 0.3333
Epoch 3/1000
 - 5s - loss: 7.6912 - acc: 0.5213 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 4/1000
 - 5s - loss: 8.0438 - acc: 0.4991 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 5/1000
 - 5s - loss: 7.8352 - acc: 0.5139 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 6/1000
 - 5s - loss: 7.4173 - acc: 0.5398 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 7/1000
 - 5s - loss: 7.5694 - acc: 0.5278 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 8/1000
 - 5s - loss: 7.2992 - acc: 0.5454 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 9/1000
 - 5s - loss: 7.5019 - acc: 0.5333 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 10/1000
 - 5s - loss: 7.9413 - acc: 0.5065 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 11/1000
 - 5s - loss: 7.5223 - acc: 0.5315 - val_los

In [0]:
scores = model.evaluate(test_images_arr, y_testOneHot)
scores[1]

300/300 [==============================] - 1s 2ms/step


0.7166666658719381

In [0]:

prediction = model.predict_classes(test_images_arr)
pd.crosstab(y_test.iloc[:,2],prediction,
            rownames=['label'],colnames=['predict'])

predict,0,1
label,,
0,201,37
1,48,14


In [0]:
#### classification report

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


names = ["Healthy eyes", "DR Disease"]
print(classification_report(y_test.iloc[:,2], prediction, target_names = names))
#print(confusion_matrix(y_test, prediction, labels=[0,1]))


              precision    recall  f1-score   support

Healthy eyes       0.81      0.84      0.83       238
  DR Disease       0.27      0.23      0.25        62

    accuracy                           0.72       300
   macro avg       0.54      0.54      0.54       300
weighted avg       0.70      0.72      0.71       300

